In [ ]:
# --- Standard library ---
import os
import glob
import shutil
import warnings

# --- Scientific / numeric ---
import numpy as np
import xarray as xr

# --- Geospatial ---
import rasterio
import rioxarray
from rasterio.transform import Affine
import rvt.default
import rvt.vis

# --- Plotting / progress ---
import matplotlib.pyplot as plt
from tqdm import tqdm

# --- Misc settings ---
warnings.filterwarnings("ignore")
cfg_datapath = '/scratch-3/vmarijn/MassBalanceMachine/../data/'

## RGI:

In [ ]:
path_geotiff = os.path.join(cfg_datapath,
                            'GLAMOS/topo/RGI_v6_11/geotiff_meters_lv95/')

rhone_rgi = "RGI60-11.01238"
rhone_path = os.path.join(path_geotiff, f"{rhone_rgi}.tif")

# Open DEM
rhone = rioxarray.open_rasterio(rhone_path).squeeze()

# Plot
plt.figure(figsize=(8, 6))
rhone.plot(cmap="terrain")
plt.title(f"DEM of Glacier {rhone_rgi}", fontsize=13)
plt.xlabel("Easting [m]")
plt.ylabel("Northing [m]")
plt.show()

### Compute skyview factor:

In [ ]:
# -------------------
# Paths & parameters
# -------------------
path_geotiff = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11",
                            "geotiff_meters_lv95/")
path_out = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11", "svf_nc/")
if os.path.exists(path_out):
    shutil.rmtree(path_out)
os.makedirs(path_out, exist_ok=True)

# RVT parameters (tune as needed)
SVF_N_DIR = 16
SVF_R_MAX = 10
SVF_NOISE = 0
ASVF_LEVEL = 1
ASVF_DIR = 315


def process_dem_to_netcdf(dem_path: str, out_path: str):
    # --- open with rasterio for authoritative geoinfo ---
    with rasterio.open(dem_path) as src:
        transform: Affine = src.transform
        crs = src.crs
        width, height = src.width, src.height
        # pixel sizes (a > 0, e < 0 for north-up rasters)
        pix_w = float(transform.a)
        pix_h = abs(float(transform.e))
        # center-of-pixel coordinates from transform
        x = transform.c + (np.arange(width) + 0.5) * pix_w
        y = transform.f + (np.arange(height) +
                           0.5) * transform.e  # note e is negative

        # read DEM as array (band 1)
        dem = src.read(1)
        no_data = src.nodata

    if max(pix_w, pix_h) < 0.0001:
        raise ValueError(
            "DEM appears to be in degrees; compute SVF on a metric CRS first.")

    # --- compute SVF / ASVF / OPNS (RVT expects one horizontal resolution) ---
    out = rvt.vis.sky_view_factor(
        dem=dem,
        resolution=pix_w,  # meters
        compute_svf=True,
        compute_asvf=True,
        compute_opns=True,
        svf_n_dir=SVF_N_DIR,
        svf_r_max=SVF_R_MAX,
        svf_noise=SVF_NOISE,
        asvf_level=ASVF_LEVEL,
        asvf_dir=ASVF_DIR,
        no_data=no_data)

    svf_arr = out["svf"]
    asvf_arr = out["asvf"]
    opns_arr = out["opns"]

    # --- build Dataset with correct LV95 x/y coords ---
    da_svf = xr.DataArray(svf_arr,
                          dims=("y", "x"),
                          coords={
                              "y": y,
                              "x": x
                          },
                          name="svf",
                          attrs={
                              "description": "Sky-View Factor",
                              "units": "unitless"
                          })
    da_asvf = xr.DataArray(asvf_arr,
                           dims=("y", "x"),
                           coords={
                               "y": y,
                               "x": x
                           },
                           name="asvf",
                           attrs={
                               "description": "Anisotropic SVF",
                               "units": "unitless",
                               "asvf_level": ASVF_LEVEL,
                               "asvf_dir": ASVF_DIR
                           })
    da_opns = xr.DataArray(opns_arr,
                           dims=("y", "x"),
                           coords={
                               "y": y,
                               "x": x
                           },
                           name="opns",
                           attrs={
                               "description": "Positive Openness",
                               "units": "radians"
                           })

    ds_out = xr.Dataset(data_vars={
        "svf": da_svf,
        "asvf": da_asvf,
        "opns": da_opns
    },
                        coords={
                            "x": x,
                            "y": y
                        },
                        attrs={
                            "source_dem": os.path.basename(dem_path),
                            "no_data":
                            no_data if no_data is not None else np.nan,
                            "crs":
                            str(crs) if crs is not None else "EPSG:2056",
                            "rvt_svf_n_dir": SVF_N_DIR,
                            "rvt_svf_r_max": SVF_R_MAX,
                            "rvt_svf_noise": SVF_NOISE,
                        })

    # compressed NetCDF (no fixed chunksizes to avoid tiny rasters failing)
    encoding = {
        v: {
            "zlib": True,
            "complevel": 4,
            "dtype": "float32"
        }
        for v in ds_out.data_vars
    }
    try:
        ds_out.to_netcdf(out_path, engine="h5netcdf", encoding=encoding)
    except Exception:
        try:
            ds_out.to_netcdf(out_path, engine="netcdf4", encoding=encoding)
        except Exception:
            ds_out.to_netcdf(out_path)  # scipy fallback (no compression)


# -------------------
# Driver
# -------------------
def main():
    tif_list = sorted(glob.glob(os.path.join(path_geotiff, "RGI60-11.*.tif")))
    # tif_list = [
    #     '/scratch-3/vmarijn/MassBalanceMachine/../data/GLAMOS/topo/RGI_v6_11/geotiff_meters_lv95/RGI60-11.01238.tif'
    # ]
    print(f"Found {len(tif_list)} DEMs in {path_geotiff}")

    for dem_path in tqdm(tif_list, desc="Computing SVF → NetCDF"):
        base = os.path.splitext(
            os.path.basename(dem_path))[0]  # e.g., RGI60-11.00408
        out_nc = os.path.join(path_out, f"{base}_svf.nc")
        try:
            process_dem_to_netcdf(dem_path, out_nc)
        except Exception as e:
            print(f"Failed on {base}: {e}")

        # Check bounds
        # # GeoTIFF bounds (authoritative)
        # with rasterio.open(dem_path) as src:
        #     print("GeoTIFF bounds:", src.bounds)

        # # NetCDF bounds from coords
        # ds = xr.open_dataset(out_nc)
        # print("NetCDF x:", float(ds.x.min()), "→", float(ds.x.max()))
        # print("NetCDF y:", float(ds.y.min()), "→", float(ds.y.max()))
        # ds.close()


if __name__ == "__main__":
    main()


In [ ]:
# --- Path to one of your generated SVF NetCDFs ---
path_svf = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11/svf_nc",
                        f"{rhone_rgi}_svf.nc")

# --- Load dataset ---
ds = xr.open_dataset(path_svf)

# get x and y bounds of the glacier
x_min, x_max = float(ds.x.min()), float(ds.x.max())
y_min, y_max = float(ds.y.min()), float(ds.y.max())

print(f"x: {x_min} to {x_max}")
print(f"y: {y_min} to {y_max}")

# --- Plot ---
fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

# SVF
ds["svf"].plot(ax=axes[0], cmap="viridis", vmin=0, vmax=1)
axes[0].set_title("Sky-View Factor (SVF)")
axes[0].set_xlabel("x (m)")
axes[0].set_ylabel("y (m)")

# ASVF
if "asvf" in ds:
    ds["asvf"].plot(ax=axes[1], cmap="viridis", vmin=0, vmax=1)
    axes[1].set_title("Anisotropic SVF (ASVF)")
    axes[1].set_xlabel("x (m)")
else:
    axes[1].set_visible(False)

# Positive Openness
if "opns" in ds:
    ds["opns"].plot(ax=axes[2], cmap="terrain")
    axes[2].set_title("Positive Openness (OPNS)")
    axes[2].set_xlabel("x (m)")
else:
    axes[2].set_visible(False)

plt.suptitle(f"{os.path.basename(path_svf)}", fontsize=14)
plt.show()

### Reproject to Lat/Lon:

In [ ]:
# -------- paths --------
path_nc_lv95 = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11",
                            "svf_nc/")  # input: LV95 x/y .nc
path_nc_ll = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11",
                          "svf_nc_latlon/")  # output: WGS84 lat/lon .nc
os.makedirs(path_nc_ll, exist_ok=True)

# empty the output folder? (set True if you want a clean slate)
EMPTY_OUTPUT_FIRST = True
if EMPTY_OUTPUT_FIRST and os.path.exists(path_nc_ll):
    shutil.rmtree(path_nc_ll)
    os.makedirs(path_nc_ll, exist_ok=True)

SRC_CRS = "EPSG:2056"  # LV95 (meters)
DST_CRS = "EPSG:4326"  # WGS84 lat/lon

# Variables we expect (only reproject those that exist)
DATA_VARS = ["svf", "asvf", "opns"]


def _affine_from_coords(x, y):
    """
    Build an Affine transform from 1D x/y center coordinates.
    Assumes x increases to the right; y typically decreases downward.
    """
    resx = float(np.median(np.diff(x.values)))
    resy = float(np.median(np.diff(y.values)))
    pix_h = -abs(resy)  # north-up convention
    x0 = float(x.values[0]) - resx / 2.0
    y0 = float(y.values[0]) + abs(resy) / 2.0
    return Affine(resx, 0.0, x0, 0.0, pix_h, y0)


def reproject_file_to_latlon(nc_in: str, nc_out: str):
    # Open/close cleanly so we don't leave files locked
    with xr.open_dataset(nc_in) as ds:
        # Check coords
        if ("x" not in ds.coords) or ("y" not in ds.coords):
            raise ValueError(
                f"{os.path.basename(nc_in)} is missing x/y coords.")

        x = ds["x"]
        y = ds["y"]
        transform = _affine_from_coords(x, y)

        vars_present = [v for v in DATA_VARS if v in ds.data_vars]
        if not vars_present:
            raise ValueError(
                f"No expected data variables {DATA_VARS} in {os.path.basename(nc_in)}"
            )

        data_ll = {}
        first_var = vars_present[0]

        def mk_rio_da(da_var):
            da = da_var.astype("float32")
            da = da.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
            da = da.rio.write_crs(SRC_CRS, inplace=True)
            da = da.rio.write_transform(transform, inplace=True)
            return da

        # Template grid from first variable
        template_ll = mk_rio_da(ds[first_var]).rio.reproject(DST_CRS)

        # Reproject all present vars and align to template
        for v in vars_present:
            da_src = mk_rio_da(ds[v])
            da_ll = template_ll if v == first_var else da_src.rio.reproject_match(
                template_ll)
            da_ll = da_ll.rename({"x": "lon", "y": "lat"})
            data_ll[v] = da_ll

        ds_out = xr.Dataset(
            data_vars=data_ll,
            coords={
                "lat": data_ll[first_var]["lat"],
                "lon": data_ll[first_var]["lon"]
            },
            attrs={
                "source_file": os.path.basename(nc_in),
                "original_crs": SRC_CRS,
                "output_crs": DST_CRS,
            },
        )
        # carry over useful attrs if present
        for key in ["rvt_svf_n_dir", "rvt_svf_r_max", "rvt_svf_noise"]:
            if key in ds.attrs:
                ds_out.attrs[key] = ds.attrs[key]

    # Write compressed NetCDF (prefer h5netcdf/netcdf4; fallback to scipy)
    encoding = {
        v: {
            "zlib": True,
            "complevel": 4,
            "dtype": "float32"
        }
        for v in data_ll.keys()
    }
    try:
        ds_out.to_netcdf(nc_out, engine="h5netcdf", encoding=encoding)
    except Exception:
        try:
            ds_out.to_netcdf(nc_out, engine="netcdf4", encoding=encoding)
        except Exception:
            ds_out.to_netcdf(nc_out)  # no compression fallback


def main():
    files = sorted(glob.glob(os.path.join(path_nc_lv95, "RGI60-11.*_svf.nc")))
    print(f"Found {len(files)} LV95 SVF files.")

    for f in tqdm(files, desc="Reprojecting SVF to lat/lon (.nc)"):
        base = os.path.basename(f).replace("_svf.nc", "")
        out_path = os.path.join(path_nc_ll, f"{base}_svf_latlon.nc")
        try:
            reproject_file_to_latlon(f, out_path)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()

In [ ]:
# Path to one of your reprojected SVF files
path_svflatlon = os.path.join(cfg_datapath, "GLAMOS/topo/RGI_v6_11",
                              "svf_nc_latlon", f"{rhone_rgi}_svf_latlon.nc"
                              #"RGI60-11.00001_svf_latlon.zarr"
                              )

# --- Load dataset ---
ds = xr.open_dataset(path_svflatlon)
print(ds)  # see variables, coords, attrs

# --- Extract variables (some glaciers may miss one or two) ---
svf = ds.get("svf")
asvf = ds.get("asvf")
opns = ds.get("opns")

# Compare to OGGM DEM
path_xr_grids = os.path.join(cfg_datapath,
                             "GLAMOS/topo/RGI_v6_11/xr_masked_grids/")
ds_oggm = xr.open_zarr(path_xr_grids + rhone_rgi + '.zarr')

# --- Plot ---
fig, axes = plt.subplots(1, 4, figsize=(15, 5), constrained_layout=True)

# SVF
if svf is not None:
    svf.plot(ax=axes[0], cmap="viridis", vmin=0, vmax=1)
    axes[0].set_title("Sky-View Factor (SVF)")
else:
    axes[0].set_visible(False)

# ASVF
if asvf is not None:
    asvf.plot(ax=axes[1], cmap="viridis", vmin=0, vmax=1)
    axes[1].set_title("Anisotropic SVF (ASVF)")
else:
    axes[1].set_visible(False)

# Positive Openness
if opns is not None:
    opns.plot(ax=axes[2], cmap="terrain")
    axes[2].set_title("Positive Openness (OPNS)")
else:
    axes[2].set_visible(False)

ds_oggm.masked_elev.plot(ax=axes[3])
axes[3].set_title("OGGM DEM")

# Common labels
for ax in axes:
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

plt.suptitle(os.path.basename(path_svflatlon), fontsize=13)
plt.show()

## SGI:

In [ ]:
path_geotiff = os.path.join(cfg_datapath,
                            'GLAMOS/topo/SGI2020/DEMs_geotiff_lv95/')

aletsch_rgi = "B36-26"
aletsch_path = os.path.join(path_geotiff, f"{aletsch_rgi}.tif")

# Open DEM
aletsch = rioxarray.open_rasterio(aletsch_path).squeeze()

# Plot
plt.figure(figsize=(8, 6))
aletsch.plot(cmap="terrain")
plt.title(f"DEM of Glacier {aletsch_rgi}", fontsize=13)
plt.xlabel("Easting [m]")
plt.ylabel("Northing [m]")
plt.show()

### Compute skyview factor:

In [ ]:
# -------------------
# Paths & parameters
# -------------------
path_geotiff = os.path.join(cfg_datapath,
                            "GLAMOS/topo/SGI2020/DEMs_geotiff_lv95")
path_out = os.path.join(cfg_datapath, "GLAMOS/topo/SGI2020/svf_nc/")
if os.path.exists(path_out):
    shutil.rmtree(path_out)
os.makedirs(path_out, exist_ok=True)

# RVT parameters (tune as needed)
SVF_N_DIR = 16
SVF_R_MAX = 10
SVF_NOISE = 0
ASVF_LEVEL = 1
ASVF_DIR = 315

# -------------------
# Driver
# -------------------
def main():
    tif_list = sorted(glob.glob(os.path.join(path_geotiff, "*.tif")))
    # tif_list = [
    #     '/scratch-3/vmarijn/MassBalanceMachine/../data/GLAMOS/topo/RGI_v6_11/geotiff_meters_lv95/RGI60-11.01238.tif'
    # ]
    print(f"Found {len(tif_list)} DEMs in {path_geotiff}")

    for dem_path in tqdm(tif_list, desc="Computing SVF → NetCDF"):
        base = os.path.splitext(
            os.path.basename(dem_path))[0]  # e.g., RGI60-11.00408
        out_nc = os.path.join(path_out, f"{base}_svf.nc")
        try:
            process_dem_to_netcdf(dem_path, out_nc)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()

### Reproject to lat/lon:

In [ ]:
# -------- paths --------
path_nc_lv95 = os.path.join(cfg_datapath, "GLAMOS/topo/SGI2020",
                            "svf_nc/")  # input: LV95 x/y .nc
path_nc_ll = os.path.join(cfg_datapath, "GLAMOS/topo/SGI2020",
                          "svf_nc_latlon/")  # output: WGS84 lat/lon .nc
os.makedirs(path_nc_ll, exist_ok=True)

# empty the output folder? (set True if you want a clean slate)
EMPTY_OUTPUT_FIRST = True
if EMPTY_OUTPUT_FIRST and os.path.exists(path_nc_ll):
    shutil.rmtree(path_nc_ll)
    os.makedirs(path_nc_ll, exist_ok=True)

SRC_CRS = "EPSG:2056"  # LV95 (meters)
DST_CRS = "EPSG:4326"  # WGS84 lat/lon

# Variables we expect (only reproject those that exist)
DATA_VARS = ["svf", "asvf", "opns"]


def main():
    files = sorted(glob.glob(os.path.join(path_nc_lv95, "*_svf.nc")))
    print(f"Found {len(files)} LV95 SVF files.")

    for f in tqdm(files, desc="Reprojecting SVF to lat/lon (.nc)"):
        base = os.path.basename(f).replace("_svf.nc", "")
        out_path = os.path.join(path_nc_ll, f"{base}_svf_latlon.nc")
        try:
            reproject_file_to_latlon(f, out_path)
        except Exception as e:
            print(f"Failed on {base}: {e}")


if __name__ == "__main__":
    main()